### Try to use LinearMap with ismutating=true, fail

In [ ]:
using LinearMaps
using IterativeSolvers

In [ ]:
function g(y::AbstractVector,x::AbstractVector) 
    y = [1 2;3 5]*x
    return y
end

S = LinearMap(g,2;ismutating=true)
b = [4.0, 3.3]
x = [2.0, 2.0]
gmres!(x,S,b)

### Try to use LinearMap with ismutating=false, works

In [ ]:
function f(x::AbstractVector) 
    return [1 2;3 5]*x
end

S = LinearMap(f,2;ismutating=false)
b = [4.0, 3.3]
gmres(S,b)

In [ ]:
[1 2;3 5]\[4.,3.3]

### Construct block matrix with functions, works

In [ ]:
m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]

[m1 m2;m3 m4]\b

In [ ]:
function al(x::AbstractVector)

m1 = [ 0.533442  0.367231  0.217876;
 0.937095  0.838996  0.38914;
 0.399906  0.409811  0.571862]
m2 = [ 0.515455  0.912171;
 0.197663  0.165777;
 0.3626    0.735575]
m3 = [ 0.763043  0.36013   0.779911; 
 0.676889  0.262959  0.0735239]
m4 = [ 0.981832  0.695422;
 0.906051  0.432737]

    
    function f(x::AbstractVector) 
        return m1*x
    end
    function g(x::AbstractVector) 
        return m2*x
    end
    function m(x::AbstractVector) 
        return m3*x
    end
    function n(x::AbstractVector) 
        return m4*x
    end 
    
    # has to do the matrix mul manually to return 1d output
    out = zeros(5)
    out[1:3] = f(x[1:3]) + g(x[4:5])
    out[4:5] = m(x[1:3]) + n(x[4:5])
    
    return out
end
S = LinearMap(al,5;ismutating=false)    
    
b = [ 0.71537 
 0.37217 
 0.728864
 0.72988 
 0.881618]
gmres(S,b)

### Test SaddleSystem

In [1]:
include("../src/SaddlePointSystems.jl")
using SaddlePointSystems

In [2]:
A = rand(80,80)
A⁻¹ = inv(A)
B₁ᵀ = rand(80,12)
B₂ = rand(12,80)
M = rand(12,12)
M⁻¹ = inv(M)
G₁ᵀ = rand(12,12)
G₂ = rand(12,12)
T₁ᵀ = rand(12,12)
T₂ = rand(12,12)

O32 = zeros(80,12)
O23 = zeros(12,80)
O22 = zeros(12,12)

S = [A O32 B₁ᵀ O32; O23 M T₁ᵀ G₁ᵀ; B₂ -T₂ O22 O22; O23 G₂ O22 O22]

rċ = rand(80)
ru̇ = rand(12)
rf = rand(12)
rλ = rand(12)
b = [rċ;ru̇;rf;rλ];

In [3]:
@time S\b

  0.770074 seconds (450.54 k allocations: 22.751 MiB, 1.22% gc time)


116-element Array{Float64,1}:
   8.46438 
  -2.76243 
 -10.2549  
 -10.2308  
   3.90973 
  13.7152  
   6.13831 
   0.651673
  -3.1685  
   3.6872  
   6.65511 
  -7.9352  
  -6.82713 
   ⋮       
  13.3305  
 -38.1967  
  -1.76671 
   8.06871 
  53.3821  
 -22.077   
 -27.3896  
 -12.5014  
  49.3291  
 -16.325   
 -14.0624  
  -1.6058  

In [4]:
norm(S*(S\b)-b)

2.6943423061098805e-13

In [5]:
rank(S) == 116

true

In [6]:
ċ = zeros(80)
u̇ = zeros(12)
f = zeros(12)
λ = zeros(12)

St = SaddleSystem((ċ, u̇, f, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M⁻¹, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

bt = (rċ, ru̇, rf, rλ);

  2.614475 seconds (4.06 M allocations: 194.884 MiB, 2.24% gc time)
  0.000110 seconds (126 allocations: 7.531 KiB)
  0.000285 seconds (126 allocations: 7.531 KiB)
  0.000062 seconds (126 allocations: 7.531 KiB)
  0.000053 seconds (126 allocations: 7.531 KiB)
  0.000044 seconds (126 allocations: 7.531 KiB)
  0.000036 seconds (126 allocations: 7.531 KiB)
  0.000035 seconds (126 allocations: 7.531 KiB)
  0.000032 seconds (126 allocations: 7.531 KiB)
  0.000032 seconds (126 allocations: 7.531 KiB)
  0.000032 seconds (126 allocations: 7.531 KiB)
  0.000039 seconds (126 allocations: 7.531 KiB)
  0.000030 seconds (126 allocations: 7.531 KiB)
  0.000032 seconds (126 allocations: 7.531 KiB)
  0.000033 seconds (126 allocations: 7.531 KiB)
  0.000029 seconds (126 allocations: 7.531 KiB)
  0.000029 seconds (126 allocations: 7.531 KiB)
  0.000033 seconds (126 allocations: 7.531 KiB)
  0.000029 seconds (126 allocations: 7.531 KiB)
  0.000033 seconds (126 allocations: 7.531 KiB)
  0.000028 seconds (

In [7]:
@time aa,bb,cc,dd = St\bt

  0.328337 seconds (152.50 k allocations: 7.521 MiB, 2.50% gc time)


([8.46438, -2.76243, -10.2549, -10.2308, 3.90973, 13.7152, 6.13831, 0.651673, -3.1685, 3.6872  …  -3.65509, -11.6134, -1.22999, 10.0879, 5.75357, 8.35732, -5.47727, 7.44171, 6.05012, 0.175919], [2.78267, 3.0264, 1.34898, -0.720454, -1.24274, -3.33729, 3.22079, -3.02026, -0.433406, 0.676699, -1.88255, 1.72172], [0.678006, -2.97102, -13.014, 0.565243, -4.87884, 13.4307, -4.19, 4.08066, -7.65534, -7.0742, 6.23931, 4.84731], [13.3305, -38.1967, -1.76671, 8.06871, 53.3821, -22.077, -27.3896, -12.5014, 49.3291, -16.325, -14.0624, -1.6058])

In [8]:
x = [aa;bb;cc;dd]
norm(S*x-b)

9.509459582923624e-13